In [3]:
import pickle

#helper functionsto dump/load pickle files
def dump(obj,filename):
    filehandler = open(filename,"wb")
    pickle.dump(obj,filehandler)
    filehandler.close()

def load(filename):
    file = open(filename,'rb')
    obj = pickle.load(file)
    file.close()
    return obj

In [4]:
#helper function used from 
def getCleanText(text, lower = True):
    text = str(text)
    parse_text = BeautifulSoup(text).get_text()
    
    letters_only = re.sub(r'http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', 
                          parse_text, flags=re.MULTILINE)
    letters_only = re.sub('^(1?[0-9]|2[0-3]):[0-5][0-9]$','time',letters_only)
    letters_only = re.sub('(name|NAME)\d{1,}','username',letters_only)   
    letters_only = re.sub("[^a-zA-Z0-9\.:']",  
                      " ",                   
                      letters_only)
    letters_only = letters_only.replace('\n'," ")
    letters_only = letters_only.replace('\r'," ")
    letters_only = re.sub('[.]{2,}', '. ', letters_only) 
    if lower:
        letters_only = letters_only.lower()
    words = CountVectorizer(stop_words='english').build_tokenizer()(letters_only)
    meaningful_words = [ w for w in words if len(w)> 0 and len(w)<20]
    clean_text = " ".join(meaningful_words)
    return clean_text

#### Loads data

In [6]:
obj = load('../dumps/df_sel_entities_list.pkl')
print(len(obj))
print(obj[:5])

[['PepsiCo Inc.', 'Elise Amendola', 'Pepsi', '3Q', 'File AP Photo', 'N.Y. PepsiCo Inc.', 'The Associated Press', 'Mobile & Apps FILE', 'File AP'], ['PepsiCo Inc.', 'Elise Amendola', 'Pepsi', '3Q', 'File AP Photo', 'N.Y. PepsiCo Inc.', 'The Associated Press', 'Mobile & Apps FILE', 'File AP'], ['the District of Columbia', 'Chrysler', 'McDonald', 'Coca-Cola', '" Executive Creative Director Ashley Sword', 'Stellar Novellas', 'Stellar Scholars', 'Roundtable', 'Stellar Romance'], ['Crisps', 'Coca-Cola', 'Walkers'], ['Coca-Cola', 'Instagram']]


#### Separates data in categories based on entities found

In [7]:
pepsiIdx=[];
cokeIdx=[];
bothIdx=[];
i=0;
for o in obj: # runs thourgh lists in list
    if(o[0] != "<UNK>"):
        foundCoke = False;
        foundPepsi = False;
        for ent in o:
            ent = str(ent).lower()
            if "coke" in ent or "coca" in ent or "cola" in ent:
                foundCoke = True;
            if "pepsi" in ent or "pepsico" in ent:
                foundPepsi = True;
        if foundCoke and foundPepsi:
            bothIdx.append(i)
        elif foundCoke:
            cokeIdx.append(i)
        elif foundPepsi:
            pepsiIdx.append(i)
    i=i+1;

#### Validation and putting everything together

In [8]:
print(pepsiIdx[:10])

[0, 1, 16, 18, 19, 21, 22, 23, 36, 37]


In [9]:
print(cokeIdx[:10])

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [10]:
print(bothIdx[:10])

[24, 25, 76, 93, 157, 158, 186, 191, 196, 211]


In [11]:
category = ["None"]*len(obj)
len(category)
for i in pepsiIdx:
    category[i] = "Pepsi"
for i in cokeIdx:
    category[i] = "Coke"
for i in bothIdx:
    category[i] = "Both"
print(len(category))
print(category[:5])

18340
['Pepsi', 'Pepsi', 'Coke', 'Coke', 'Coke']


#### Merge with main DF

In [12]:
df = load("../dumps/df_sel_entities_sentiment.pkl")
print(df.shape)
df.head(2)

(18340, 18)


,Unnamed: 0,index,Sound Bite Text,Title,Post Type,Media Type,Source Type,Domain,Richness,Interests,Positive Objects,Negative Objects,Topic_top,Topic_id,Topic_raw,Topic_name,entities,Sentiment
0,0,0,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN,6,6,6:0.987499999969,terms14.png,elise amendola_file ap_the associated press_mo...,0.147243
1,1,1,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN,6,6,6:0.98908045974,terms14.png,elise amendola_file ap_the associated press_mo...,0.137346


In [13]:
df = df.assign(Category = category)
print(df.shape)
df.head(2)

(18340, 19)


,Unnamed: 0,index,Sound Bite Text,Title,Post Type,Media Type,Source Type,Domain,Richness,Interests,Positive Objects,Negative Objects,Topic_top,Topic_id,Topic_raw,Topic_name,entities,Sentiment,Category
0,0,0,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN,6,6,6:0.987499999969,terms14.png,elise amendola_file ap_the associated press_mo...,0.147243,Pepsi
1,1,1,"Mobile & Apps FILE - In this July 9, 2015, fil...",PepsiCo beats 3Q profit forecasts,Original,No Media,Blogs,mercedsunstar.com,1.0,NaN,NaN,NaN,6,6,6:0.98908045974,terms14.png,elise amendola_file ap_the associated press_mo...,0.137346,Pepsi


#### Dumps Data

In [14]:
dump(df, "../dumps/df_sel_entities_sentiment_category.pkl")